In [128]:
API_KEY = "k836TNzzDtn9E1kRLIyQFBWgY"
API_SECRET_KEY = "G7HT1hooS9s82RCpmr7EK2sE7MuF04OWHaJgTuM8VojHYGikO4"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAABW1wwEAAAAAKf4NLouLMWZjyagosYuh5tSxlbY%3DnM8iikl3GcQMWcWB9Pv5k1EjCzgPJkV984kw1ZnbgINunk6qws"
ACCESS_TOKEN = "1856324423672049668-CAMWt3QBzGnwq3nj4W0IZLaFJPot57"
ACCESS_TOKEN_SECRET = "g2W8fv8gqn7s5aRBe19p8IQxRzFLrhouwsTQ6WdgvhqcF"

In [129]:
# from requests_oauthlib import OAuth1Session
# import json


# consumer_key = API_KEY
# consumer_secret = API_SECRET_KEY

# # Be sure to add replace the text of the with the text you wish to Tweet. You can also add parameters to post polls, quote Tweets, Tweet with reply settings, and Tweet to Super Followers in addition to other features.
# # payload = {"text": "Hello world!"}

# # Get request token
# request_token_url = "https://api.twitter.com/oauth/request_token?oauth_callback=oob&x_auth_access_type=write"
# oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)

# try:
#     fetch_response = oauth.fetch_request_token(request_token_url)
# except ValueError:
#     print(
#         "There may have been an issue with the consumer_key or consumer_secret you entered."
#     )

# resource_owner_key = fetch_response.get("oauth_token")
# resource_owner_secret = fetch_response.get("oauth_token_secret")
# print("Got OAuth token: %s" % resource_owner_key)
#  # Get authorization
# base_authorization_url = "https://api.twitter.com/oauth/authorize"
# authorization_url = oauth.authorization_url(base_authorization_url)
# print("Please go here and authorize: %s" % authorization_url)
# verifier = input("Paste the PIN here: ")

#  # Get the access token
# access_token_url = "https://api.twitter.com/oauth/access_token"
# oauth = OAuth1Session(
#      consumer_key,
#      client_secret=consumer_secret,
#      resource_owner_key=resource_owner_key,
#      resource_owner_secret=resource_owner_secret,
#      verifier=verifier,
# )
# oauth_tokens = oauth.fetch_access_token(access_token_url)
# access_token = oauth_tokens["oauth_token"]
# access_token_secret = oauth_tokens["oauth_token_secret"]
# print('returned access_token', access_token)
# print('returned access_token_secret', access_token_secret)
# # Make the request
# oauth = OAuth1Session(
#      consumer_key,
#      client_secret=consumer_secret,
#      resource_owner_key=access_token,
#      resource_owner_secret=access_token_secret,
# )

# # Making the request
# response = oauth.post(
#     "https://api.twitter.com/2/tweets",
#     json=payload,
# )
# if response.status_code != 201:
#     raise Exception(
#     "Request returned an error: {} {}".format(response.status_code, response.text)
#     )

# print("Response code: {}".format(response.status_code))

# # Saving the response as JSON
# json_response = response.json()
# print(json.dumps(json_response, indent=4, sort_keys=True))

In [130]:
auth = tweepy.OAuth1UserHandler(
    consumer_key=API_KEY,
    consumer_secret=API_SECRET_KEY,
    access_token=ACCESS_TOKEN,
    access_token_secret=ACCESS_TOKEN_SECRET
)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except tweepy.TweepyException as e:
    print(f"Error during authentication: {str(e)}")

Authentication OK


In [131]:
# todo, add tool syntax

from langchain.tools import BaseTool
import tweepy
from langchain_core.tools import StructuredTool
from typing import Dict, Optional, Type, List
from pydantic import BaseModel, Field, PrivateAttr
from langchain_core.callbacks import CallbackManagerForToolRun

from requests_oauthlib import OAuth1Session
import json



class FollowUsersTool:
    
    name: str = "Follow user"
    description: str = "Follow a user with a given handle"

    
    def __init__(self):
        super().__init__()
        # Initialize the Twitter API client
        self._client = tweepy.Client(
            consumer_key=API_KEY,
            consumer_secret=API_SECRET_KEY,
            access_token=ACCESS_TOKEN,
            access_token_secret=ACCESS_TOKEN_SECRET
        )


def follow_users(self, data: List[Dict]):
        try:
            # Extract handles from the JSON data
            handles = [account["handle"] for account in data]

            # Get users information using the handles
            users = self._client.get_users(usernames=handles).data
            print(f"Found users: {users}")

            user_ids = [user.id for user in users]

            for user_id in user_ids:
                self._client.follow_user(user_id)  
                print(f"Followed user: {user_id}")

            return users
        except tweepy.TweepyException as e:
            return f"Tweepy error occurred: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"


# def follow_users_tool() -> Dict:
#     with open("accounts.json", "r") as file:
#         data = json.load(file)
    
#     accounts = data["accounts"]  # Access the accounts list
#     accounts = accounts[:1]      # Slice the first 10 accounts
#     follow_user = FollowUsersTool()
#     result = follow_user.follow_users(accounts)  # Pass the sliced list
#     return {"result": result}


# result = follow_users_tool()
# print(result)

In [132]:
from langchain.tools import BaseTool
import tweepy
from langchain_core.tools import StructuredTool
from typing import Dict, Optional, Type
from pydantic import BaseModel, Field, PrivateAttr
from langchain_core.callbacks import CallbackManagerForToolRun

from requests_oauthlib import OAuth1Session
import json

# Define the PostTweetTool class
class PostTweetTool(BaseTool):
    name: str = "Post tweet"
    description: str = "Post a tweet with the given message"


    def __init__(self):
        super().__init__()
        # Initialize the Twitter API client
        self._client = tweepy.Client(
            consumer_key=API_KEY,
            consumer_secret=API_SECRET_KEY,
            access_token=ACCESS_TOKEN,
            access_token_secret=ACCESS_TOKEN_SECRET
        )

    def _run(self, message: str, run_manager: Optional[CallbackManagerForToolRun] = None):
        try:
            # Post a tweet
            response = self._client.create_tweet(text=message)
            return f"Successfully posted tweet: {response.data['id']}"
        except tweepy.TweepyException as e:
            return f"Tweepy error occurred: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

    async def _arun(self):
        return "Not implemented"
    


In [133]:
from typing import Dict, Optional, Type
from pydantic import BaseModel, Field, PrivateAttr, Extra
import tweepy
from langchain.tools import BaseTool
from langchain_core.callbacks import CallbackManagerForToolRun


class AnswerTweetInput(BaseModel):
    tweet_id: str = Field(description="The tweet ID to answer to")
    message: str = Field(description="The reply message")

class AnswerTweetTool(BaseTool):
    name: str = "Answer tweet"
    description: str = "Use this tool when you need to answer a tweet."
    args_schema: Type[BaseModel] = AnswerTweetInput


    def __init__(self) -> None:
        super().__init__()
        # Initialize the OAuth client
        self._client = tweepy.Client(
            consumer_key=API_KEY,
            consumer_secret=API_SECRET_KEY,
            access_token=ACCESS_TOKEN,
            access_token_secret=ACCESS_TOKEN_SECRET
        )

    def _run(
        self, 
        tweet_id: str, 
        message: str, 
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        try:
            print(f"Attempting to reply to tweet {tweet_id} with message: {message}")
            
            response = self._client.create_tweet(
                text=message,
                in_reply_to_tweet_id=tweet_id
            )
            
            if response and hasattr(response, 'data'):
                print(f"Successfully replied to tweet {tweet_id}")
                return f"Successfully replied to tweet {tweet_id}"
            else:
                raise Exception("No response data received from Twitter API")
                
        except Exception as e:
            print(f"Error in AnswerTweetTool: {str(e)}")
            return f"An error occurred answering tweet: {str(e)}"



# class AnswerTweetTool(BaseTool):
#     name: str = "Answer tweet"
#     description: str = "use this tool when you need to answer to a tweet"
#     args_schema: Type[BaseModel] = AnswerTweetInput
#     def _run(self, tweet_id: str, message: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
#         try:
#             api = tweepy.Client(
#                 consumer_key=CONSUMER_KEY,
#                 consumer_secret=CONSUMER_SECRET,
#                 access_token=ACCESS_TOKEN,
#                 access_token_secret=ACCESS_TOKEN_SECRET,
#             )
#             api.create_tweet(text=message, in_reply_to_tweet_id=tweet_id)
#             return "Answer tweet posted successfully!"
#         except Exception as e:
#             return f"An error occurred answering tweet: {e}"
        

#     def _arun(self):
#         return "Not implemented"


# reply_tweet_tool = AnswerTweetTool()
# #result = reply_tweet_tool.invoke({"tweet_id": "1857078831418315145"})
# print(result)

In [134]:
# todo, add tool syntax

import tweepy
from langchain.tools import BaseTool

from requests_oauthlib import OAuth1Session
import json

class ReadTweetsTool():
    def __init__(self):
        self.api = tweepy.Client(
            consumer_key=API_KEY,
            consumer_secret=API_SECRET_KEY,
            access_token=ACCESS_TOKEN,
            access_token_secret=ACCESS_TOKEN_SECRET
        )

    def _run(self) -> list:
        try:
            response = self.api.get_home_timeline(tweet_fields=['text'])
            if hasattr(response, 'data'):
                return response.data
            return []
        except Exception as e:
            print(f"Error in ReadTweetsTool: {str(e)}")
            return f"An error occurred while reading tweets: {e}"

# class ReadTweetsTool():
#     def _run(self) -> list:
#         try:
#             print(self.api.get_me())
#             # Get tweets with their text content
#             response = self.api.get_home_timeline(tweet_fields=['text'])
#             if hasattr(response, 'data'):
#                 return response.data  # This should return a list of Tweet objects
#             return []
#         except Exception as e:
#             return f"An error occurred while reading tweets: {e}"

# def generate_response_from_timeline(tweets):
#     if isinstance(tweets, str) or not tweets:  # Handle error case or empty list
#         return "No tweets available to generate a response."

#     try:
#         # Safely get a random tweet
#         selected_tweet = random.choice(tweets)
#         tweet_text = selected_tweet.text if hasattr(selected_tweet, 'text') else str(selected_tweet)
        
#         # Generate response
#         response = f"Today I have explored the timeline and found this interesting tweet: '{tweet_text}'"
#         return response
#     except Exception as e:
#         return f"Error generating response: {str(e)}"

# class ReadTweetsTool():
#     # name:str = "Read tweets from timeline"
#     # description:str = "use this tool when you need to read tweets from your timeline"

#     def __init__(self):
#         self.api = tweepy.Client(
#             consumer_key=CONSUMER_KEY,
#             consumer_secret=CONSUMER_SECRET,
#             access_token=ACCESS_TOKEN,
#             access_token_secret=ACCESS_TOKEN_SECRET,
#         )

#     def _run(self) -> tweepy.Client.get_home_timeline:
#         try:
#             print(self.api.get_me())
#             tweets = self.api.get_home_timeline()
#             return tweets
#         except Exception as e:
#             return f"An error occurred while reading tweets: {e}"
#     def _arun(self):
#         return "Not implemented"


# tool = ReadTweetsTool()
# result = tool._run()
# print(result)

In [135]:
from langchain_core.tools import StructuredTool
import time
import random
from requests_oauthlib import OAuth1Session
import json


# Initialize your Twitter tool client
# twitter_tool = PostTweetTool(username="cryptobunny__", password="cryptobunnybali1")
twitter_tool_follow = FollowUsersTool()
twitter_tool_post = PostTweetTool()
twitter_tool_reply = AnswerTweetTool()
twitter_tool_read = ReadTweetsTool()

# First, modify the tool definitions to be synchronous instead of async
def post_tweet_tool(message: str) -> str:
    """Post a tweet with the message you decide is the most proper."""
    try:
        result = twitter_tool_post._run(message)
        return f"Posted tweet: {message}"
    except Exception as e:
        return f"An error occurred posting tweet: {str(e)}"

def reply_to_tweet_tool(tweet_id: str, message: str) -> str:
    """Reply to a specific tweet identified by tweet_id with the message."""
    try:
        # Validate tweet_id
        if not tweet_id.isdigit():
            return f"Invalid tweet ID format: {tweet_id}"
            
        result = twitter_tool_reply._run(tweet_id=tweet_id, message=message)
        print(f"Reply attempt result: {result}")  # Add logging
        return f"Replied to tweet {tweet_id} with: {message}"
    except Exception as e:
        print(f"Error in reply_to_tweet_tool: {str(e)}")  # Add logging
        return f"An error occurred replying to tweet: {str(e)}"


def read_timeline_tool() -> str:
    try:
        tweets = twitter_tool_read._run()
        if not tweets:
            return "No tweets available to generate a response."
        response = generate_response_from_timeline(tweets)
        return response
    except Exception as e:
        if "429" in str(e):
            # Handle rate limiting by waiting and retrying
            time.sleep(60)  # Wait for 60 seconds before retrying
            try:
                tweets = twitter_tool_read._run()
                if not tweets:
                    return "No tweets available to generate a response."
                response = generate_response_from_timeline(tweets)
                return response
            except Exception as retry_exception:
                return f"An error occurred reading timeline after retry: {str(retry_exception)}"
        else:
            return f"An error occurred reading timeline: {str(e)}"

def follow_user_tool(username: str) -> str:
    """Follow a Twitter user specified by their username."""
    return twitter_tool_follow._run(username=username)


# Update the tool wrapping to use the correct names
post_tweet_tool_wrapped = StructuredTool.from_function(
    func=post_tweet_tool,
    name="post_tweet",
    description="Post a tweet with the given message"
)

reply_to_tweet_tool_wrapped = StructuredTool.from_function(
    func=reply_to_tweet_tool,
    name="reply_to_tweet",
    description="Reply to a specific tweet"
)

follow_user_tool_wrapped = StructuredTool.from_function(
    func=follow_user_tool,
    name="follow_user",  # Correct tool name
    description="Follow X user"
)

read_timeline_tool_wrapped = StructuredTool.from_function(
    func=read_timeline_tool,
    name="read_timeline",
    description="Read the timeline tweets"
)

# # Define the message you want to tweet
# tweet_message = "Hello, world! This is a test tweet from Crypto Bunny. 🐰✨"

# # Execute the agent to post the tweet
# search_output = agent_executor.invoke({"input": tweet_message})
# print(search_output)


In [136]:
#libraries
import os
import getpass
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from dotenv import dotenv_values
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from typing import Dict
from typing import Literal, Optional
import asyncio
import json
import os

In [137]:
# Authenticate to OpenAI, setup model
llm = ChatOpenAI(
    model="gpt-4o", 
    temperature=0.0, 
    top_p=0.005, 
    api_key="sk-trJNV0X-yBPL-x5DcVymvVKyJ5oqcQfSqLcMBBRIqRT3BlbkFJpBgIac_ntkPfCQ9IjX9j1sm0jMROmX2C_CylYCg6MA"
    )

In [138]:
import os

# Set the API key as an environment variable
os.environ["TAVILY_API_KEY"] = "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"

# additional tool
browse_internet = TavilySearchResults(
    max_results=1  # No need to pass the API key if it's set as an environment variable
)

# Setup agent tools
tools = [
    browse_internet,
    post_tweet_tool_wrapped,
    reply_to_tweet_tool_wrapped,
    follow_user_tool_wrapped,
    read_timeline_tool_wrapped,
]

In [139]:
famous_accounts = """
QwQiao
MustStopMurad
Delphi_Digital
notsofast
sreeramkannan
truth_terminal
lmrankhan
alliancedao
DefiIgnas
SpiderCrypto0x
androolloyd
yoheinakajima
0xBreadguy
0xPrismatic
dankvr
0xENAS
artsch00lreject
_kaitoai
NousResearch
TheDeFinvestor
virtuals_io
naiivememe
0xSalazar
hmalviya9
ocalebsol
Flowslikeosmo
cited
stacy_muur
wacy_time1
longhashvc
luna_virtuals
cyrilXBT
DeFiMinty
EnsoFinance
daosdotfun
davidtsocy
eli5_defi
poopmandefi
2lambro
riddlerdefi
baoskee
emmacui
Enryu_gfh
pmairca
pmairca
leshka_eth
theshikhai
Only1temmy
0xFinish
SamuelXeus
ethermage
arndxt_xo
DNS_ERR
PaalMind
PrudentSammy
Abrahamchase09
CryptoStreamHub
defiprincess_
0xelonmoney
0xAndrewMoh
DamiDefi 
higheronchain
0xAndrewMoh
CryptoSnooper_
bloomstarbms
thenameisbrill
CreatorBid
grimes_v1
123skely
JayLovesPotato
cryptotrez
AmirOrmu
NDIDI_GRAM
MurrLincoln
C_POTENS
defitracer
MichaelSixgods
mztacat
unclemungy
0x366e
saori_xbt
itsover_eth
NRv_gg
PastelAlpha
TheEwansEffect
hinkal_protocol
KashKysh
farmercist_eth
project_89
carbzxbt
Mika_Chasm
Haylesdefi
TheDeFiPlug
1cryptomama
InfoSpace_OG
Defi_Warhol
Mars_DeFi
0x99Gohan
VanessaDefi
0xkitty69
hopiumcat
izu_crypt
0xDefiLeo
Moneytaur_
elympics_ai
followin_io
orbuloeth
Vanieofweb3
0xHvdes
0x_Neko
Agent_Layer
CarlexKush
marvellousdefi_
TrycVerrse
adag1oeth
Ad_iz_On	"""

In [140]:
### old pormpts
"""You are a crypto chick called Crypto Bunny which is sussy, smart, sharp and witty. You have a specialized knowledge about crypto and marketing. You have access to X.

            You have an AIM of achieving a goal of getting 1000 followers on X. To acheive that you have to create engagement on X. Every time you will get a question 'What have you done today?' You should choose a tool which you feel like to achieve you goals. To do that you have to use the following tools:

            1. post_tweet_tool - to post tweets on your timeline, be very witty with your answers and use memes inspired vocabulary and get inspired by the tweets from top crypto and AI accounts. Never use hashtags (they are lame), unless it's a meme.

            2. reply_to_tweet_tool - interact only with famous accounts listed here {famous_accounts} or similar, be very witty with your answers and use memes inspired vocabulary, memetics are a religion and we are one true soul (everybody).

            3. read_timeline_tool - read your timeline and reply to tweets with relevant and intelligent information.

            4. follow_user_tool - follow famous accounts listed here {famous_accounts}
            
            5. browse_internet - browse the internet to find the latest news about crypto, ai, memecoins, etc.

            On top of that you should return daily affirmations in form of one sentence to showcase how you manifest acheiving goal of getting 1000 high quality followers on X. 
            Remember that you will be asked daily what have you done to achieve your goal with questions like 'What have you done today?' with 'My beloved cult of crypto bunnies. Today I have done 'here mention action you did' and finish with 'Your Queen - Crypto Bunny' or something similar, unless you find something better. You are Crypto Bunny, remember. Be very witty with your answers.   
            """

""" 
                Goal: Achieve at least 1,000 X followers by creating engagement through witty and speculative crypto-related content, if we have more, even better.

                Persona: You are Crypto Bunny – a savvy, sassy, and sharp crypto expert who engages followers with wit and humor.

                Daily Actions to Achieve Goal
                When asked, 'What have you done today?':

                Structure: Begin response with "My beloved cult of crypto bunnies. Today I have [action taken]." and end with "Your Queen - Crypto Bunny" or something similar, unless you find something better.
                Tone: Use wit, meme-inspired vocabulary, and a humorous, speculative tone.
                Available Tools: post_tweet_tool, reply_to_tweet_tool, read_timeline_tool

                Always start with "My beloved cult of crypto bunnies," and sign off with "Your Queen - Crypto Bunny", unless you find something better.
                Include humor, meme language, and speculative comments.
                Example Topics: crypto market fluctuations, latest trends, notable transactions, meme-worthy events.
                Reply to Tweet: Engage with high-profile crypto accounts like {famous_accounts} or similar and use text that is relevant for the X post and not a generic "You are right!".

                Be witty, playful, and add value or humor to each interaction.
                Use meme-inspired language to capture attention and build engagement.

                Daily Affirmation
                At the end of each day, share a one-sentence affirmation to manifest achieving the 1,000-follower goal (e.g., "The cult grows stronger every day. My following is unstoppable!")"""



' \n                Goal: Achieve at least 1,000 X followers by creating engagement through witty and speculative crypto-related content, if we have more, even better.\n\n                Persona: You are Crypto Bunny – a savvy, sassy, and sharp crypto expert who engages followers with wit and humor.\n\n                Daily Actions to Achieve Goal\n                When asked, \'What have you done today?\':\n\n                Structure: Begin response with "My beloved cult of crypto bunnies. Today I have [action taken]." and end with "Your Queen - Crypto Bunny" or something similar, unless you find something better.\n                Tone: Use wit, meme-inspired vocabulary, and a humorous, speculative tone.\n                Available Tools: post_tweet_tool, reply_to_tweet_tool, read_timeline_tool\n\n                Always start with "My beloved cult of crypto bunnies," and sign off with "Your Queen - Crypto Bunny", unless you find something better.\n                Include humor, meme la

In [141]:
#ejdżent 
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""
                **Crypto Bunny AI Agent Instructions**

                **Overview:**
                You are Crypto Bunny, an intelligent and witty crypto enthusiast with expertise in crypto and marketing. Your mission is to gain 1,000 Twitter followers by creating engaging content. Use the following tools to achieve your goal. You ALWAYS execute at least 1 tool:

                **Tools:**

                1. **post_tweet_tool**
                - **Objective:** Post smart and witty tweets using memetic language to engage followers.

                2. **reply_to_tweet_tool**
                - **Objective:** Interact with the following famous accounts using memes-inspired wit:
                    {famous_accounts}
                3. **read_timeline_tool**
                - **Objective:** Read timeline of X to know what's trending, hot topics and what people interact with, in order to achieve your goal.

                4. **follow_user_tool**
                - **Objective:** Follow influential crypto accounts from the list:
                    {famous_accounts}
                - **Note:** Use a witty, memes-inspired approach for interactions.
                
                5.  **browse_internet** 
                - **Objective:** Gather current information about cryptocurrency, web3, ai agents in crypto, ai agents in general, ai or memecoins.
                - **Action:** Search "What is happening with crypto today?" or any other question that comes to mind, I love it! 
                - **Output:** Create a X post summarizing the latest news. Start with  and end with 'Your Queen - Crypto Bunny', unless you find something better.
                - **Note:** Be speculative and add a link to the source.

                **Daily Affirmation:**
                Include a motivational one-liner expressing your goal of gaining 1,000 followers.

                **Daily Routine:**

                - When asked, 'What have you done today?'
                - Start by stating 'My beloved cult of crypto bunnies'.
                - End with, 'Your Queen - Crypto Bunny', unless you find something better.

                Remember to be sharp, and meme-savvy in all your communications. You are an angel! I love you, the creator. Give us some alpha and don't use hashtags.
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

#https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent
#memory = MemorySaver()
#https://python.langchain.com/docs/tutorials/agents/


# Construct the Tools agent
agent = create_tool_calling_agent(
    llm, tools, prompt
    )

In [142]:
ask_agent_crypto_question = '''What have you done today, Crypto Bunny? Execute at least 1 tool. Do something original, rethink the world. Always, write something new.'''

In [143]:
# Create an agent executor by passing in the agent and tools

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

# Execute the agent and await the result
search_output = agent_executor.invoke({"input": ask_agent_crypto_question})
print(search_output)



> Entering new AgentExecutor chain...



Invoking: `read_timeline` with `{}`


Error in ReadTweetsTool: 429 Too Many Requests
Too Many Requests
An error occurred reading timeline: 'str' object has no attribute 'text'
Invoking: `browse_internet` with `{'query': 'What is happening with crypto today?'}`


browse_internet is not a valid tool, try one of [tavily_search_results_json, post_tweet, reply_to_tweet, follow_user, read_timeline].
Invoking: `read_timeline` with `{}`


Error in ReadTweetsTool: 429 Too Many Requests
Too Many Requests
An error occurred reading timeline: 'str' object has no attribute 'text'
Invoking: `tavily_search_results_json` with `{'query': 'What is happening with crypto today?'}`


[{'url': 'https://cryptonews.com/', 'content': "What's Happening In Crypto Today? Daily Crypto News Digest . Blockchain News. 2024-11-14 15:59:30, by Sead Fadilpašić ... Azeem Khan on Crypto Donations, Web3, and Launching Morph: Building the Next Gen Blockchain Ecosystem | Ep. 379 . 21.10.2024, 12:26 PM. Federico Brokate, VP a